In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
from sklearn import model_selection
from PIL import Image
from torchvision import transforms
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import timm
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from torch.utils.data import DataLoader
import time
import copy

In [1]:


data = yf.download('EURJPY=X', start='2001-01-03', end='2021-03-09')

data.head()


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2003-01-23,126.980003,127.110001,126.519997,126.879997,126.879997,0
2003-01-24,126.900002,127.760002,126.800003,127.489998,127.489998,0
2003-01-27,127.589996,128.940002,127.489998,128.440002,128.440002,0
2003-01-28,128.460007,128.720001,128.020004,128.470001,128.470001,0
2003-01-29,128.460007,128.619995,127.889999,128.279999,128.279999,0


In [2]:
data['Up'] = 0
# future_num日後の終値の上下を予測
future_num = 1
for i in range(len(data)-1):
    if data['Close'][i] < data['Close'][i+future_num]:
        data['Up'][i] = 1

C:\Users\yt-in\AppData\Local\Temp\ipykernel_4012\710805594.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Up'][i] = 1


In [5]:
data['ImageName'] =  ['./DataFolder/' + str(data.index[i].year)+ '-'  + str(data.index[i].month) + '-' + str(data.index[i].day) + '.png' for i in range(len(data))]

In [8]:
img_create = 1
if img_create == 1:
    seq_len = 20

    data.fillna(0)
    #data.reset_index(inplace=True)
    data['Date2'] = data.index
    data['Date2'] = data['Date2'].map(mdates.date2num)
    for i in range(0, len(data)-1):
        c = data.iloc[i:i + int(seq_len) , :]
        if len(c) == int(seq_len):
            # Date,Open,High,Low,Adj Close,Volume
            target = ['Date2','Open','High','Low','Close']
            pngfile = data['ImageName'][i + int(seq_len)-2]

            my_dpi = 96
            # Create my own `marketcolors` to use with the `nightclouds` style:
            mc = mpf.make_marketcolors(up='#77d879',down='#db3f3f',inherit=True)

            # Create a new style based on `nightclouds` but with my own `marketcolors`:
            s  = mpf.make_mpf_style(base_mpf_style='nightclouds',marketcolors=mc,gridstyle = '')

            fig = mpf.figure(style=s,figsize=(48 / my_dpi, 48 / my_dpi), dpi=my_dpi)
            ax1 = fig.add_subplot(1,1,1)

            ax1.grid(False)
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.xaxis.set_visible(False)
            ax1.yaxis.set_visible(False)
            ax1.axis('off')

            mpf.plot(c[target], ax=ax1,volume=False,type='candle',update_width_config=dict(candle_linewidth=1))
            fig.savefig(pngfile,  pad_inches=0, transparent=False, bbox_inches="tight")

            plt.close(fig)


    print("Converting olhc to candlestik finished.")

Converting olhc to candlestik finished.


In [13]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(data['ImageName'].values, data['Up'].values, test_size=0.1,random_state=1)

In [15]:
X = []
Y = []
X_val = []
Y_val = []

transform = transforms.Compose([transforms.ToTensor()])

#for image_file,label in zip(x_train,y_train):
for image_file,label in zip(x_train,y_train):
    try:
        image = Image.open(image_file)
        image = image.convert('RGB')
        image = transform(image)
        #image = np.asarray(image)
        X.append(image)
        Y.append(label)
    except FileNotFoundError as e:
        print(e)


for image_file,label in zip(x_val,y_val):
    try:
        image = Image.open(image_file)
        image = image.convert('RGB')
        image = transform(image)
        X_val.append(image)
        Y_val.append(label)
    except FileNotFoundError as e:
        print(e)

[Errno 2] No such file or directory: './DataFolder/2003-1-24.png'
[Errno 2] No such file or directory: './DataFolder/2003-2-10.png'
[Errno 2] No such file or directory: './DataFolder/2003-1-31.png'
[Errno 2] No such file or directory: './DataFolder/2021-3-8.png'
[Errno 2] No such file or directory: './DataFolder/2003-1-28.png'
[Errno 2] No such file or directory: './DataFolder/2003-2-3.png'
[Errno 2] No such file or directory: './DataFolder/2003-1-29.png'
[Errno 2] No such file or directory: './DataFolder/2003-2-7.png'
[Errno 2] No such file or directory: './DataFolder/2003-2-6.png'
[Errno 2] No such file or directory: './DataFolder/2003-2-14.png'
[Errno 2] No such file or directory: './DataFolder/2003-2-4.png'
[Errno 2] No such file or directory: './DataFolder/2003-1-23.png'
[Errno 2] No such file or directory: './DataFolder/2003-2-5.png'
[Errno 2] No such file or directory: './DataFolder/2003-1-27.png'
[Errno 2] No such file or directory: './DataFolder/2003-1-30.png'
[Errno 2] No suc

In [17]:
class EUR_Dataset(Dataset):
    def __init__(self, image, label_list, phase=None):

        self.transform = transforms.Compose([transforms.ToTensor()])
        self.image = image
        self.label_list = torch.tensor(label_list,dtype=torch.long)
        self.phase = phase


    def __getitem__(self,index):
        # index番目の画像を読み込み、前処理を行う
        image = self.image[index]
        # index番目のラベルを取得する
        label = self.label_list[index]

        return image, label

    def __len__(self):
        return len(self.image)

In [18]:
train_dataset = EUR_Dataset(X,Y,phase='train')
total_samples = len(train_dataset)
print(total_samples)
val_dataset = EUR_Dataset(X_val,Y_val,phase='val')
val_samples = len(val_dataset)
print(val_samples)

4214
469


In [30]:

model = models.resnet18(pretrained=False)

num_ftrs = model.fc.in_features

model.fc  = nn.Sequential(
                      nn.Linear(num_ftrs ,2))

In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

criterion = nn.CrossEntropyLoss()


# Observe that all parameters are being optimized
optimizer_ft = optim.AdamW(model.parameters())

batchsize = 16

loss_list = []
val_loss_list = []
# DataLoaderのインスタンス作成
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batchsize, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batchsize, shuffle=True)

# データローダーを辞書として格納する
dataloaders_dict = {
    'train' : train_dataloader,
    'val' : val_dataloader
}

cpu


In [34]:
Train_loss = []
Val_loss = []
Train_acc = []
Val_acc = []


def train_model(model, criterion, optimizer,  num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    #best_acc = 0.0
    best_loss = 1000.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase

        for phase in ['train', 'val']:
            running_loss = 0.0
            running_corrects = 0

            # 学習モード、検証モードの切替
            if phase == 'train':
                model.train()
            else:
                model.eval()

            #for i,data in enumerate(trainloader):
            i = 0
            data_num = 0
            for inputs, labels in dataloaders_dict[phase]:
                i += 1
                #for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, axis=1)

                    #print(preds)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    data_num += inputs.size(0)

            epoch_loss = running_loss/data_num
            epoch_acc = running_corrects.double()/data_num


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'train':
                Train_loss.append(epoch_loss)
                Train_acc.append(epoch_acc)
            else:
                Val_loss.append(epoch_loss)
                Val_acc.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [35]:
model = train_model(model,criterion,optimizer_ft,num_epochs=20)

Epoch 0/19
----------
train Loss: 0.5645 Acc: 0.7100
val Loss: 0.3841 Acc: 0.8252

Epoch 1/19
----------


KeyboardInterrupt: 